**Fuente de Datos:**
1.   [Inflacion] (https://www.indec.gob.ar/)
2.   [Dolar Oficial] y [Salario Minimo]  (https://datos.gob.ar/)  

**Librerias, variables con informacion de los dataset y inicio de los dataframes**

In [ ]:
#Librerias
import pandas as pd
from google.colab import data_table

#Variable con ubicacion y nombre de los archivos
path = "/content/drive/MyDrive/Proyecto #4/"
archivo_salario_minimo = "indice_salario_minimo.csv"
archivo_dolar = "datos-tipo-cambio-usd-futuro-dolar-frecuencia-diaria.csv"
archivo_inflacion = "serie_ipc_divisiones.csv"

#Convertimos en dataframes.
df_1 = pd.read_csv(path+archivo_salario_minimo)
df_2 = pd.read_csv(path+archivo_dolar)
df_3 = pd.read_csv(path+archivo_inflacion, encoding="ISO-8859-1", on_bad_lines = "skip", sep= ";")#Utilizamos varios parametros para que pandas intente leer este desastre.
                                                                                                  #La "ISO-8859-1" le decimos a pandas que este archivo contiene caracteres y acentos que no tiene el lenguaje ingles.
                                                                                                  #Con las otras dos parametros le decimos que las lineas problematicas sean ignoradas y que use como separacion de columnas ";".

**Analizamos los dataframes**

In [ ]:
#Con .info() verificamos la informacion de los dataframes
df_1.info()
#df_2.info()
#df_3.info()

#Analizamos los dataframes
data_table.DataTable(df_1)
#data_table.DataTable(df_2)
#data_table.DataTable(df_3)

**Limpieza dataframe #1**

In [ ]:
#Filtramos el dataframe con las columnas que vamos a utilizar
df_1 = df_1[["salario_minimo_vital_movil_mensual", "indice_tiempo"]].copy() #Utilizamos .copy() para decirle a pandas que esta es una copia del original, aunque guardemos todo en la misma variable
df_1["indice_tiempo"] = pd.to_datetime(df_1["indice_tiempo"]) #Le avisamos a pandas que esa columna es de fechas, que la interprete como tal
df_1 = df_1[df_1["indice_tiempo"] >= "2016-12-01"].copy()

#Calculamos la diferencia mensual
df_1.reset_index(drop=True, inplace=True)  #Reseteamos el indice para que empiece desde 0
number_indice_final = 1
number_indice_inicial = 0
number_cantidad_filas = df_1["indice_tiempo"].count() - 1 #En este variable se va guardar la cantidad de filas, se utilizara para terminar el loops de while.
while number_indice_final <= number_cantidad_filas:
   df_1.loc[number_indice_final, "porcentaje mensual"] = (
                                                         (df_1.loc[number_indice_final, "salario_minimo_vital_movil_mensual"] - df_1.loc[number_indice_inicial, "salario_minimo_vital_movil_mensual"])/
                                                          df_1.loc[number_indice_inicial, "salario_minimo_vital_movil_mensual"]*100)
   number_indice_inicial += 1
   number_indice_final += 1

#Calculo de porcentaje acumulado
df_1.loc[0, "Acumulado %"] = 100
number_indice_final_1 = 1
number_indice_inicial_1 = 0
while number_indice_final_1 <= number_cantidad_filas:
   df_1.loc[number_indice_final_1, "Acumulado %"] =  (df_1.loc[number_indice_inicial_1, "Acumulado %"] * df_1.loc[number_indice_final_1, "porcentaje mensual"]/100) + df_1.loc[number_indice_inicial_1, "Acumulado %"]
   number_indice_inicial_1 += 1
   number_indice_final_1 += 1

#Dejamos los primeros 2 decimales
df_1["porcentaje mensual"]=df_1["porcentaje mensual"].round(2)
df_1["Acumulado %"]=df_1["Acumulado %"].round(2)

#Modificamos la columna donde se encuentran la fecha, dejamos solamente año y mes.
df_1["indice_tiempo"] = df_1["indice_tiempo"].dt.strftime("%Y-%m")
df_1 = df_1.loc[df_1["indice_tiempo"] >= "2017-01"].copy() #Filtramos con determinada fecha en adelante

#Borramos la columna de la fecha y reseteamos el indice.
df_1.drop("indice_tiempo", axis=1, inplace=True)
df_1 = df_1.reset_index(drop=True)

#Cambiamos los nombres de las columnas.
df_1.rename(columns=
                    {"salario_minimo_vital_movil_mensual" : "Salario mínimo, vital y móvil", "porcentaje mensual" : "Salario - Variación mensual (%)", "Acumulado %" : "Salario - % acumulado"}, inplace=True)


**Limpiezaz dataframe #2**


In [ ]:
#Filtramos el dataframe
df_2["indice_tiempo"] = pd.to_datetime(df_2["indice_tiempo"]) #Convertimos a fecha
df_2["Day"] = df_2["indice_tiempo"].dt.day #Creamos una nueva columna con solamente los dias para dps filtrar con las fechas de forma mensual del dia 1 de cada mas.
df_2["indice_tiempo"] = df_2["indice_tiempo"].dt.strftime("%Y-%m") #Dejamos solamente año y mes
df_2 = df_2.loc[(df_2["indice_tiempo"] >= "2016-12") & (df_2["Day"]== 1)]#Filtramos por fecha y dia
df_2 = df_2.reset_index(drop=True) #Reseteamos el indice

#Calculo de diferencia, utilizando while para que pase por todas las filas de la columna
number_indice_final =  1
number_indice_inicial = 0
number_cantidad_filas = df_2["indice_tiempo"].count() - 1
while number_indice_final <= number_cantidad_filas:
    df_2.loc[number_indice_final, "Dolar % mensual"] = (
                                                       (df_2.loc[number_indice_final, "tipo_cambio_bna_vendedor"] - df_2.loc[number_indice_inicial, "tipo_cambio_bna_vendedor"])/
                                                        df_2.loc[number_indice_inicial, "tipo_cambio_bna_vendedor"]*100)
    number_indice_final +=1
    number_indice_inicial +=1
#Calculo de porcentaje acumulado
df_2.loc[0, "% Acumulado"] = 100
number_indice_final =  1
number_indice_inicial = 0
while number_indice_final <= number_cantidad_filas:
   df_2.loc[number_indice_final, "% Acumulado"] =  (df_2.loc[number_indice_inicial, "% Acumulado"] * df_2.loc[number_indice_final, "Dolar % mensual"]/100) + df_2.loc[number_indice_inicial, "% Acumulado"]
   number_indice_inicial += 1
   number_indice_final += 1

#Dejamos los primeros 2 decimales
df_2["Dolar % mensual"]=df_2["Dolar % mensual"].round(2)
df_2["% Acumulado"]=df_2["% Acumulado"].round(2)

#Filtramos columnas y fecha
df_2 = df_2.loc[df_2["indice_tiempo"] >= "2017-01"]
df_2 = df_2[["tipo_cambio_bna_vendedor", "Dolar % mensual", "% Acumulado"]]
df_2 = df_2.reset_index(drop=True) #Reset indice

#Cambiamos los nombres a las columnas
df_2.rename(columns=
                 {"tipo_cambio_bna_vendedor": "Dólar oficial", "Dolar % mensual" : "Dólar - Variación mensual (%)", "% Acumulado" : "Dólar - % acumulado"}, inplace=True)

**Limpieza dataframe #3**

In [ ]:
#Filtramos el dataframe
df_3 = df_3[(df_3["Region"]=="Nacional") & (df_3["Descripcion"]=="NIVEL GENERAL")] #filtramos las filas que vamos a necesitar
df_3 = df_3[["Periodo","Indice_IPC", "v_m_IPC"]] ##Dejamos las columnas que vamos a usar
df_3["Periodo"] = df_3["Periodo"].astype(str) #Convertimos de int a string
df_3["año"] = df_3["Periodo"].str[:4]  #Creamos una columna vamos a guardar los primeros 4 strings de esta columna
df_3["mes"] = df_3["Periodo"].str[4:]  #Creamos otra columna donde se guardara los ultimos 2
df_3["indice_tiempo"] = df_3["año"] + "-" +df_3["mes"] #Creamos una columna donse sumaremos los strings para crear la fecha, ademas de agregar el separador.
df_3["indice_tiempo"] = df_3["indice_tiempo"] + "-01" #Le agregamos el primer dia de cada mes
df_3.drop(["Periodo", "año", "mes"], axis=1, inplace=True) #Borramos las columnas que no vamos a usar
df_3["indice_tiempo"] = pd.to_datetime(df_3["indice_tiempo"])
df_3 = df_3.loc[df_3["indice_tiempo"] >= "2017-01"]

#Reemplazamos los "," por ".", esto se debe a que python reconoce como separador de decimal al "." no la ",". Se puede dejar con "," pero en este caso vamos hacer calculos.
df_3["Indice_IPC"] = df_3["Indice_IPC"].str.replace(",", ".")
df_3["v_m_IPC"] = df_3["v_m_IPC"].str.replace(",", ".")

#Cambiamos el formato a numero flotante
df_3["Indice_IPC"] = df_3["Indice_IPC"].astype(float)
df_3["v_m_IPC"] = df_3["v_m_IPC"].astype(float)
df_3 = df_3.reset_index(drop=True) #reseteamos indice

#Cambiamos los nombres a las columnas
df_3.rename(columns=
                 {"indice_tiempo": "Fecha", "Indice_IPC" :"Inflación acumulada (%)", "v_m_IPC": "Inflación mensual (%)"}, inplace=True)


**Union, orden y guardado del dataframes**

In [ ]:
#Unimos los dataframes
df = pd.concat([df_1, df_2, df_3], axis=1)

#Creamos una variable la ultima fecha del dataframe, se utlizara para el informe final.
cantidad_de_filas = df["Fecha"].count() - 1
ultima_fecha = df.loc[cantidad_de_filas, "Fecha"]
ultima_fecha = str(ultima_fecha)
ultima_fecha = ultima_fecha[:10]

#Convertimos fecha como indice
df.set_index("Fecha", inplace=True)

#Ordenamos las columnas
columns = ["Salario mínimo, vital y móvil", "Salario - Variación mensual (%)", "Salario - % acumulado", "Dólar oficial", "Dólar - Variación mensual (%)", "Dólar - % acumulado",
           "Inflación mensual (%)", "Inflación acumulada (%)"]
df = df[columns]

#Lo guardamos en formato csv en la nube de google
#df.to_csv(path+"Proyecto_4_1.csv")

**Calculos de diferencia entre salarios, inflacion y dolar**

In [ ]:
#Creamos variables con los datos mas recientes
salario = df.loc[:,"Salario - % acumulado"].iloc[cantidad_de_filas]
inflacion = df.loc[:,"Inflación acumulada (%)"].iloc[cantidad_de_filas]
dolar = df.loc[:, "Dólar - % acumulado"].iloc[cantidad_de_filas]

#Calculamos la diferencia
diferencia_salario_x_inflacion = (inflacion - salario) / salario * 100
diferencia_salario_x_dolar = (dolar - salario) / salario * 100

#Redondeamos los decimales
diferencia_salario_x_inflacion = diferencia_salario_x_inflacion.round(2)
diferencia_salario_x_dolar = diferencia_salario_x_dolar.round(2)


Conclusion

In [ ]:
print("Ultima actualizacion:", ultima_fecha)
print("* Salario acumulado:", salario, "%")
print("* Inflacion acumulada:", inflacion, "%")
print("* Dolar acumulado:", dolar, "%")
print("\nEl salario mínimo creció un",diferencia_salario_x_inflacion,"% menos que la inflación y un",diferencia_salario_x_dolar ,"% menos que el dólar oficial en el mismo período (2017-01-01 a",ultima_fecha+")"", \
lo que evidencia una pérdida del poder adquisitivo frente a ambos indicadores.")

Ultima actualizacion: 2025-03-01
* Salario acumulado: 3926.35 %
* Inflacion acumulada: 8353.3158 %
* Dolar acumulado: 6755.45 %

El salario mínimo creció un 112.75 % menos que la inflación y un 72.05 % menos que el dólar oficial en el mismo período (2017-01-01 a 2025-03-01), lo que evidencia una pérdida del poder adquisitivo frente a ambos indicadores.
